In [1]:
import pickle
import cv2
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import glob
%matplotlib qt

In [2]:
def camera_calib(filenames,nx=9,ny=6):
    objp = np.zeros((ny*nx,3),np.float32)
    objp[:,:2] = np.mgrid[0:nx, 0:ny].T.reshape(-1,2) #here we created a nice chessboard
    
    imgpoints = []
    objpoints = []

    for fname in filenames:
        img = cv2.imread(fname)
        gray=cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
        ret, corners = cv2.findChessboardCorners(gray, (nx, ny), None)

        if ret == True:
            imgpoints.append(corners)
            objpoints.append(objp)

    ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, gray.shape[::-1],None,None)
    return (ret, mtx, dist, rvecs, tvecs)

In [16]:
def undistort(img, cam_params):
    ret, mtx, dist, rvecs, tvecs = cam_params
    return cv2.undistort(img, mtx, dist, None, mtx) 

        
filenames=glob.glob("camera_cal/calibration*.jpg")
cam_params=camera_calib(filenames)
cal_jpg = cv2.imread("camera_cal/calibration5.jpg")
cv2.imwrite('output_images/calibrate.jpg', undistort(cal_jpg,cam_params))

True

In [4]:
def convert_to_gray(img):
    return cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

def abs_sobel_thresh(img, orient='x', sobel_kernel=3, thresh=(0,255)):
    gray = convert_to_gray(img)

    if orient == 'x':
        abs_sobel = np.absolute(cv2.Sobel(gray, cv2.CV_64F, 1, 0, ksize=sobel_kernel))
    if orient == 'y':
        abs_sobel = np.absolute(cv2.Sobel(gray, cv2.CV_64F, 0, 1, ksize=sobel_kernel))
    
    scaled_sobel = np.uint8(255*abs_sobel/np.max(abs_sobel))
    binary_output = np.zeros_like(scaled_sobel)
    binary_output[(scaled_sobel >= thresh[0]) & (scaled_sobel <= thresh[1])] = 1
    return binary_output

def mag_thresh(img, sobel_kernel=3, thresh=(0, 255)):
    # Convert to grayscale
    gray = convert_to_gray(img)
    # Take both Sobel x and y gradients
    sobelx = cv2.Sobel(gray, cv2.CV_64F, 1, 0, ksize=sobel_kernel)
    sobely = cv2.Sobel(gray, cv2.CV_64F, 0, 1, ksize=sobel_kernel)
    # Calculate the gradient magnitude
    gradmag = np.sqrt(sobelx**2 + sobely**2)
    # Rescale to 8 bit
    scale_factor = np.max(gradmag)/255 
    gradmag = (gradmag/scale_factor).astype(np.uint8) 
    # Create a binary image of ones where threshold is met, zeros otherwise
    binary_output = np.zeros_like(gradmag)
    binary_output[(gradmag >= thresh[0]) & (gradmag <= thresh[1])] = 1
    return binary_output

# Define a function to threshold an image for a given range and Sobel kernel
def dir_threshold(img, sobel_kernel=9, thresh=(0, np.pi/2)):
    # Grayscale
    gray = convert_to_gray(img)
    # Calculate the x and y gradients
    sobelx = cv2.Sobel(gray, cv2.CV_64F, 1, 0, ksize=sobel_kernel)
    sobely = cv2.Sobel(gray, cv2.CV_64F, 0, 1, ksize=sobel_kernel)
    # Take the absolute value of the gradient direction, 
    # apply a threshold, and create a binary image result
    absgraddir = np.arctan2(np.absolute(sobely), np.absolute(sobelx))
    binary_output =  np.zeros_like(absgraddir)
    binary_output[(absgraddir >= thresh[0]) & (absgraddir <= thresh[1])] = 1

    # Return the binary image
    return binary_output

def Canny_threshold(img, thresh=(0,255)):
    # Grayscale
    gray = convert_to_gray(img)
    binary_output = cv2.Canny(gray,thresh[0],thresh[1])
    binary_output[binary_output==255]=1
    return binary_output
    
def hls_threshold(img, thresh=(0, 255)):
    hls = cv2.cvtColor(img, cv2.COLOR_RGB2HLS)
    # s channel is the best for this threshold
    s_channel = hls[:,:,2]
    binary_output = np.zeros_like(s_channel)
    binary_output[(s_channel > thresh[0]) & (s_channel <= thresh[1])] = 1
    return binary_output

def combine_Sobel(img, SobelX=True, SobelY=True, SobelX_params=[], SobelY_params=[]):
    gray = convert_to_gray(img)
    binary_output=np.ones_like(gray)
    
    if(SobelX):
        img, sobel_kernel, thresh = SobelX_params
        binary_output = binary_output & abs_sobel_thresh(img=img, orient='x', 
                                                         sobel_kernel=sobel_kernel, thresh=thresh)
    
    if(SobelX):
        img, sobel_kernel, thresh = SobelY_params
        binary_output = binary_output & abs_sobel_thresh(img=img, orient='y', 
                                                         sobel_kernel=sobel_kernel, thresh=thresh)
        
    return binary_output    

def combine_MagDir(img, Mag=True, Dir=True, Mag_params=[], Dir_params=[]):
    gray = convert_to_gray(img)
    binary_output=np.ones_like(gray)
    
    if(Mag):
        img, sobel_kernel, thresh = Mag_params
        binary_output = binary_output & abs_sobel_thresh(img=img, orient='x', 
                                                         sobel_kernel=sobel_kernel, thresh=thresh)
    
    if(Dir):
        img, sobel_kernel, thresh = Dir_params
        binary_output = binary_output & abs_sobel_thresh(img=img, orient='y', 
                                                         sobel_kernel=sobel_kernel, thresh=thresh)
    
    return binary_output

In [5]:
def combine_threshold(img, Canny=True, Sobel=True, MagDir=True, HLS=True,
                     Canny_params=[], Sobel_params=[], MagDir_params=[], HLS_params=[]):

    #only for resizing
    gray = convert_to_gray(img)  
    binary_output=np.zeros_like(gray)
        
    if(Canny):
        img, thresh = Canny_params
        binary_output = binary_output | Canny_threshold(img, thresh)
        canny_img = Canny_threshold(img,thresh)
        
    if(Sobel):
        img, SobelX, SobelY, SobelX_params, SobelY_params = Sobel_params
        binary_output = binary_output | combine_Sobel(img, SobelX, SobelY, SobelX_params, SobelY_params)
        
    if(MagDir):
        img, Mag, Dir, Mag_params, Dir_params = MagDir_params
        binary_output = binary_output | combine_MagDir(img, Mag, Dir, Mag_params, Dir_params)
        
    if(HLS):
        img, thresh = HLS_params
        binary_output = binary_output | hls_threshold(img,thresh)
        hls_img = hls_threshold(img,thresh)

    return binary_output

In [6]:
def get_perspective_transform(src = np.float32([[373,605], [923,605], [532, 496], [756,496]]),
                             dst = np.float32([[373,605], [923,605], [373, 487], [923,487]])):
    return cv2.getPerspectiveTransform(src,dst)

def perspective_transform(img,M):
    img_size = (img.shape[1], img.shape[0])
    return cv2.warpPerspective(img, M, img_size, flags=cv2.INTER_LINEAR)

In [7]:
def window_mask(width, height, img_ref, center,level):
    output = np.zeros_like(img_ref)
    output[int(img_ref.shape[0]-(level+1)*height):int(img_ref.shape[0]-level*height),
           max(0,int(center-width/2)):min(int(center+width/2),img_ref.shape[1])] = 1
    return output

def find_window_centroids(warped, window_width, window_height, margin):
    window_centroids = [] # Store the (left,right) window centroid positions per level
    window = np.ones(window_width) # Create our window template that we will use for convolutions
    
    # First find the two starting positions for the left and right lane by using np.sum to get the vertical image slice
    # and then np.convolve the vertical image slice with the window template 
    
    # Sum quarter bottom of image to get slice, could use a different ratio
    l_sum = np.sum(warped[int(3*warped.shape[0]/4):,:int(warped.shape[1]/2)], axis=0)
    l_center = np.argmax(np.convolve(window,l_sum))-window_width/2
    r_sum = np.sum(warped[int(3*warped.shape[0]/4):,int(warped.shape[1]/2):], axis=0)
    r_center = np.argmax(np.convolve(window,r_sum))-window_width/2+int(warped.shape[1]/2)
    
    # Add what we found for the first layer
    window_centroids.append((l_center,r_center))
    
    # Go through each layer looking for max pixel locations
    for level in range(1,(int)(warped.shape[0]/window_height)):
       # convolve the window into the vertical slice of the image
        image_layer = np.sum(warped[int(warped.shape[0]-(level+1)*window_height):int(warped.shape[0]-level*window_height),:], axis=0)
        conv_signal = np.convolve(window, image_layer)
        # Find the best left centroid by using past left center as a reference
        # Use window_width/2 as offset because convolution signal reference is at right side of window, not center of window
        offset = window_width/2
        l_min_index = int(max(l_center+offset-margin,0))
        l_max_index = int(min(l_center+offset+margin,warped.shape[1]))
        l_center = np.argmax(conv_signal[l_min_index:l_max_index])+l_min_index-offset
        # Find the best right centroid by using past right center as a reference
        r_min_index = int(max(r_center+offset-margin,0))
        r_max_index = int(min(r_center+offset+margin,warped.shape[1]))
        r_center = np.argmax(conv_signal[r_min_index:r_max_index])+r_min_index-offset
        # Add what we found for that layer
        window_centroids.append((l_center,r_center))

    return window_centroids


def draw_window_centroids(centroids, warped, window_width, window_height, margin, saving=0):
    window_centroids = centroids

    # If we found any window centers
    if len(window_centroids) > 0:

        # Points used to draw all the left and right windows
        l_points = np.zeros_like(warped)
        r_points = np.zeros_like(warped)

        # Go through each level and draw the windows 	
        for level in range(0,len(window_centroids)):
            # Window_mask is a function to draw window areas
            l_mask = window_mask(window_width,window_height,warped,window_centroids[level][0],level)
            r_mask = window_mask(window_width,window_height,warped,window_centroids[level][1],level)
            # Add graphic points from window mask here to total pixels found 
            l_points[(l_points == 255) | ((l_mask == 1) ) ] = 255
            r_points[(r_points == 255) | ((r_mask == 1) ) ] = 255

        # Draw the results
        template = np.array(r_points+l_points,np.uint8) # add both left and right window pixels together
        zero_channel = np.zeros_like(template) # create a zero color channel
        template = np.array(cv2.merge((zero_channel,template,zero_channel)),np.uint8) # make window pixels green
        warpage = np.array(cv2.merge((warped,warped,warped)),np.uint8) # making the original road pixels 3 color channels
        output = cv2.addWeighted(warpage, 1, template, 0.5, 0.0) # overlay the orignal road image with window results
            
 
    # If no window centers found, just display orginal road image
    else:
        output = np.array(cv2.merge((warped,warped,warped)),np.uint8)

    result = cv2.addWeighted(warped, 1, convert_to_gray(output), 0.1, 0)

    return result



In [8]:
# Define a class to receive the characteristics of each line detection
class Line():
    def __init__(self):
        self.first_image = True
        self.left_fitx = []
        self.right_fitx = []
        
saved_line = Line()
current_line = Line()

def fit_lanes(window_centroids, ploty):
    unzipped = zip(*window_centroids)
    leftx, rightx = unzipped

    # Fit a second order polynomial to pixel positions in each fake lane line
    left_fit = np.polyfit(ploty, leftx, 2)
    current_line.left_fitx = left_fit[0]*ploty**2 + left_fit[1]*ploty + left_fit[2]
    right_fit = np.polyfit(ploty, rightx, 2)
    current_line.right_fitx = right_fit[0]*ploty**2 + right_fit[1]*ploty + right_fit[2]

# reality checks -- starting from the second frame
def reality_checks( diff_threshold_left = 6400., diff_threshold_right = 6400. ):

    if( not(current_line.first_image) ):
        if( np.linalg.norm( current_line.left_fitx - saved_line.left_fitx ) > diff_threshold_left  ):
            current_line.left_fitx = saved_line.left_fitx
        if( np.linalg.norm( current_line.right_fitx - saved_line.right_fitx ) > diff_threshold_right ):
            current_line.right_fitx = saved_line.right_fitx
        
        # saved line = 1/3 current_line + 2/3 saved_line 
        # = 1/3 current_line + 2/9 last_line + 4/9 previous_saved_line=.., i.e. power series style smoothing
        saved_line.right_fitx = (current_line.right_fitx + 2*saved_line.right_fitx)/3.
        saved_line.left_fitx = (current_line.left_fitx + 2*saved_line.left_fitx)/3.        
        
    else:
        # first frame -- just save all
        saved_line.left_fitx = current_line.left_fitx
        saved_line.right_fitx = current_line.right_fitx
        current_line.first_image = False

In [9]:
def draw_lines(warped, left_fitx, right_fitx, Minv, undist, ploty, saving=0):
    # everything before was upside down w.r.t. cv2 coords
    ploty_inv = 720-ploty
    # Create an image to draw the lines on
    warp_zero = np.zeros_like(warped).astype(np.uint8)
    color_warp = np.dstack((warp_zero, warp_zero, warp_zero))

    # Recast the x and y points into usable format for cv2.fillPoly()
    pts_left = np.array([np.transpose(np.vstack([left_fitx, ploty_inv]))])
    pts_right = np.array([np.flipud(np.transpose(np.vstack([right_fitx, ploty_inv])))])
    pts = np.hstack((pts_left, pts_right))
        
    if(saving):        
        color_lines = np.dstack((255*warped, 255*warped, 255*warped))        
        line_image=cv2.addWeighted(color_warp, 1, color_lines, 0.3, 0)        
        cv2.polylines(color_lines, np.int_([pts_left]), False, (0,255, 0), 3)
        cv2.polylines(color_lines, np.int_([pts_right]), False, (0,0, 255), 3)
        cv2.imwrite('output_images/line_image.jpg', color_lines)
   
    imshape=warped.shape
    # Draw the lane onto the warped blank image
    cv2.fillPoly(color_warp, np.int_([pts]), (0,255, 0))

    # Warp the blank back to original image space using inverse perspective matrix (Minv)
    newwarp = perspective_transform(color_warp, Minv) 

    # Combine the result with the original image
    result = cv2.addWeighted(undist, 1, newwarp, 0.3, 0)
    return result

In [10]:
# average curvature radius
def curvature_and_offset(leftx, rightx, ploty):
    #in this array y minimum is closest to the vehicle, y max far on the road
    y_eval = np.min(ploty)

    # Define conversions in x and y from pixels space to meters
    # more meters as my perspective transform is more "squeezed"
    ym_per_pix = 60/720 # meters per pixel in y dimension
    xm_per_pix = 3.7/700 # meters per pixel in x dimension

    # Fit new polynomials to x,y in world space
    left_fit_cr = np.polyfit(ploty*ym_per_pix, leftx*xm_per_pix, 2)
    right_fit_cr = np.polyfit(ploty*ym_per_pix, rightx*xm_per_pix, 2)
    # Calculate the new radii of curvature
    left_curverad = ((1 + (2*left_fit_cr[0]*y_eval*ym_per_pix + left_fit_cr[1])**2)**1.5) / np.absolute(2*left_fit_cr[0])
    right_curverad = ((1 + (2*right_fit_cr[0]*y_eval*ym_per_pix + right_fit_cr[1])**2)**1.5) / np.absolute(2*right_fit_cr[0])
    # Now our radius of curvature is in meters
    curvature = (left_curverad + right_curverad)/2.
    # how much are we drifting to the right on our lane 
    offset = (1280*xm_per_pix - left_fit_cr[2] - right_fit_cr[2])/2.
    return curvature, offset

In [11]:
def process_image(original, saving=0):
    
    undistorted = undistort(original, cam_params)
    
    Canny_params = undistorted, (50,150)
    SobelX_params = undistorted, 7, (10,255)
    SobelY_params = undistorted, 7, (10,255)
    Sobel_params = undistorted, True, True, SobelX_params, SobelY_params
    Mag_params = undistorted, 7, (0,255)
    Dir_params = undistorted, 7, (0.65, 1.05)
    MagDir_params = undistorted, True, True, Mag_params, Dir_params
    HLS_params = undistorted, (70,256)


    thresholded = combine_threshold(undistorted, Canny=True, Sobel=True, MagDir=False, HLS=True,
                                   Canny_params=Canny_params, Sobel_params=Sobel_params, 
                                    MagDir_params=MagDir_params, HLS_params=HLS_params)
    

    M=get_perspective_transform()
    Minv=np.linalg.inv(M)
    warped = perspective_transform(thresholded,M)


    # window settings
    window_width = 50 
    window_height = 80 # Break image into 9 vertical layers since image height is 720
    margin = 100 # How much to slide left and right for searching

    centroids = find_window_centroids(warped, window_width, window_height, margin)
    centroid_fig = draw_window_centroids(centroids, warped, window_width, window_height, margin, saving)

    ploty = np.linspace(0, 719, num=9)
    fit_lanes(centroids, ploty)
    reality_checks(600, 200)

    result = draw_lines(warped,current_line.left_fitx, current_line.right_fitx, Minv, undistorted, ploty, saving)

    curvature, offset = curvature_and_offset(current_line.left_fitx, current_line.right_fitx, ploty)

    cv2.putText(result,
                'Curvature: ' + str(curvature)+' m',
                (10,20), cv2.FONT_HERSHEY_PLAIN, 1,(255,255,255),2)
    
    cv2.putText(result,
                'Offset: ' + str(offset)+' m',
                (10,40), cv2.FONT_HERSHEY_PLAIN, 1,(255,255,255),2)

        
    if(saving):
        cv2.imwrite('output_images/undistorted.jpg', undistorted)
        cv2.imwrite('output_images/thresholded.jpg', 255*thresholded)
        cv2.imwrite('output_images/warped.jpg', 255*warped)
        cv2.imwrite('output_images/result.jpg', result)
        cv2.imwrite('output_images/centroids.jpg', 50*centroid_fig)
    
    return result
    

In [14]:
from moviepy.editor import VideoFileClip
from IPython.display import HTML

# video processing
current_line.first_image = True
white_output = 'output.mp4'
clip1 = VideoFileClip("project_video.mp4")
white_clip = clip1.fl_image(process_image)
get_ipython().magic('time white_clip.write_videofile(white_output, audio=False)')

# image processing
current_line.first_image = True
image_names=glob.glob("test_images/test*.jpg") 
plt.ioff()
for iname in image_names:
    original = cv2.imread( iname )
    if 'processed' not in iname:
        processed = cv2.cvtColor(process_image(original), cv2.COLOR_RGB2BGR)
        plt.imshow(processed)
        plt.savefig('output_images'+iname[11:-4]+'_processed.jpg')
        current_line.first_image = True
     
# this image is to generate jpg output for the writeup
fig_gen = cv2.imread('test_images/test1.jpg')
proc=process_image(fig_gen, saving=1)

[MoviePy] >>>> Building video output.mp4
[MoviePy] Writing video output.mp4


100%|█████████▉| 1260/1261 [04:24<00:00,  4.49it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: output.mp4 

CPU times: user 6min 4s, sys: 9.09 s, total: 6min 13s
Wall time: 4min 29s
